In [ ]:
import sys
sys.path.append("/glade/work/juliob/myPythonTools/")
""" Now you can imprt modules in ../CASutils """

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import pandas as pd


from CASutils import zmplot_utils as zmplots
from CASutils import calendar_utils as cal
from CASutils import readdata_utils as read
from CASutils import plotposition_utils as plotpos
from CASutils import colorbar_utils as cbars
from CASutils import lineplot_utils as lineplot
from CASutils import mapplot_utils as mymaps
from CASutils import averaging_utils as avg
from CASutils import cam7_utils as cam7


"""
import zmplot_utils as zmplots
import calendar_utils as cal
import readdata_utils as read
import plotposition_utils as plotpos
import colorbar_utils as cbars
import lineplot_utils as lineplot
import mapplot_utils as mymaps
import averaging_utils as avg
import cam7_utils as cam7
"""

import dask

import importlib
importlib.reload(cam7)
importlib.reload(zmplots)
importlib.reload(cbars)

In [ ]:
mls = xr.open_dataset("/glade/campaign/cgd/cas/islas/CAM7validation/MLS/mls_h2o_latNpressNtime_3d_monthly_v5.nc")
mls = mls.rename(x='lat', y='lev', t='time')
time = pd.date_range("2004-09","2021-11",freq='M')
mls['time'] = time
mls = avg.cosweightlat(mls.H2O,-10,10)
mls = mls.groupby('time.month').mean('time')
mls = mls*18.015280/(1e6*28.964)

In [ ]:
era5 = xr.open_dataset("/glade/campaign/cgd/cas/islas/CAM7validation/ERA5/ERA5_Q_10Sto10N_1980to2020.nc")
era5 = era5.groupby('time.month').mean('time')

In [ ]:
runs_LT={'#250':'f.cam6_3_107.FLTHIST_v0a.ne30.clm5_1.001',
      '#259':'f.cam6_3_109.FLTHIST_v0b.ne30.tuningF.001',
      '#235':'f.cam6_3_105.FLTHIST_v0a.ne30.001',
      '#229':'f.cam6_3_101.FLTHIST_v0a.ne30.001',
      '#305':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3-rdg_beta0.5-vtrmi1.0.001',
      '#306':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3-rdg_beta0.5-wsubi1.0.001',
      '#307':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3-rdg_beta0.5-dcs500.001',
      '#296':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3-rdg_beta0.5.001',
      '#252':'f.cam6_3_107.FLTHIST_v0a.ne30.clm5_1_ebudget.001',
      '#270':'f.cam6_3_109.FLTHIST_v0b.ne30.tuningF_hb.001',
      '#313':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3-rdg_beta0.5-vtrmi2.0.001'}

runs_MT={'#310':'f.cam6_3_112.FCMTHIST_v0c.ne30.non-ogw-ubcT-effgw0.7.001',
         '#288':'f.cam6_3_112.FMTHIST_v0c.ne30.non-ogw-ubcT-effgw0.7.001'}

In [ ]:
#############################################
# This cell is extremely slow. Not sure why.
#############################################

basepath="/glade/scratch/juliob/processed/CAM7/jun2023/Q/"
alldat=[]
runname_LT=[]
for key in runs_LT:
    print(key)
    print(runs_LT[key])
    dat = xr.open_dataset(basepath+runs_LT[key]+'.nc')
    dat = read.fixcesmtime(dat)
    datzm = dat.mean('lon')
    dat_tropics = avg.cosweightlat(datzm.Q, -10, 10)
    dat_mon = dat_tropics.groupby('time.month').mean('time').load()
    alldat.append(dat_mon)
    runname_LT.append(key)
    
runname_LT=xr.DataArray(runname_LT, dims='run', coords=[np.arange(0,len(runname_LT),1)], name='run')
alldat_concat_LT = xr.concat(alldat, dim=runname_LT)


alldat=[]
runname_MT=[]
for key in runs_MT:
    print(key)
    print(runs_MT[key])
    dat = xr.open_dataset(basepath+runs_MT[key]+'.nc')
    dat = read.fixcesmtime(dat)
    datzm = dat.mean('lon')
    dat_tropics = avg.cosweightlat(datzm.Q, -10, 10)
    dat_mon = dat_tropics.groupby('time.month').mean('time').load()
    alldat.append(dat_mon)
    runname_MT.append(key)
    
runname_MT=xr.DataArray(runname_MT, dims='run', coords=[np.arange(0,len(runname_MT),1)], name='run')
alldat_concat_MT = xr.concat(alldat, dim=runname_MT)

In [ ]:
x1, x2, y1, y2 = plotpos.get5by5coords_zmplots()

fig = plt.figure(figsize=(16,16))

ax = cam7.plot_pre_mon(fig, mls, mls.lev, 1e-7,1.5e-6,3e-6,'MLS',x1[0],x2[0],y1[0],y2[0],cmap='precip_nowhite', paxis='lev',
                       taxis='month')
ax = cam7.plot_pre_mon(fig, era5.Q, era5.pre, 1e-7,1.5e-6,3e-6,'ERA5',x1[1],x2[1],y1[1],y2[1], cmap='precip_nowhite', paxis='pre',
                       taxis='month')

count=2

for irun in np.arange(0,alldat_concat_LT.run.size,1):
    ax = cam7.plot_pre_mon(fig, alldat_concat_LT.isel(run=irun), alldat_concat_LT.isel(run=irun).lev, 1e-7,1.5e-6,3e-6,alldat_concat_LT.run.isel(run=irun).values,
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')
    count=count+1
    
for irun in np.arange(0,alldat_concat_MT.run.size,1):
    ax = cam7.plot_pre_mon(fig, alldat_concat_MT.isel(run=irun), alldat_concat_MT.isel(run=irun).lev, 1e-7,1.5e-6,3e-6,alldat_concat_MT.run.isel(run=irun).values,
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')
    count=count+1    
    
ax = cbars.plotcolorbar(fig, 1e-7,1.5e-6,3e-6,'Q (kg/kg)', x1[1],x2[3],y1[count]-0.03,y1[count]-0.02,cmap='precip_nowhite')

fig.savefig('h2o.png', bbox_inches='tight', facecolor='white')

In [ ]:
fig = plt.figure(figsize=(16,16))

count=0
ax = cam7.plot_pre_mon(fig, alldat_concat_LT.sel(run='#259'), alldat_concat_LT.sel(run='#259').lev, 1e-7,2e-6,4e-6,alldat_concat_LT.run.sel(run='#259').values,
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

count=count+1
ax = cam7.plot_pre_mon(fig, alldat_concat_LT.sel(run='#270'), alldat_concat_LT.sel(run='#270').lev, 1e-7,2e-6,4e-6,alldat_concat_LT.run.sel(run='#270').values,
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

ax = cbars.plotcolorbar(fig, 1e-7,2e-6,4e-6,'Q (kg/kg)', x1[0],x2[1],y1[2]-0.03,y1[2]-0.02,cmap='precip_nowhite')

fig.savefig('hbdiff_impact.png', bbox_inches='tight', facecolor='white')

In [ ]:
print(alldat_concat_MT.sel(run='#288'))

In [ ]:
fig = plt.figure(figsize=(16,16))

count=0
ax = cam7.plot_pre_mon(fig, alldat_concat_MT.sel(run='#288'), alldat_concat_MT.sel(run='#288').lev, 1e-7,2e-6,4e-6,str(alldat_concat_MT.run.sel(run='#288').values)+' MT (no chem)',
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

count=count+1
ax = cam7.plot_pre_mon(fig, alldat_concat_MT.sel(run='#310'), alldat_concat_MT.sel(run='#310').lev, 1e-7,2e-6,4e-6,str(alldat_concat_MT.run.sel(run='#310').values)+' MT (chem)',
                           x1[count],x2[count],y1[count],y2[count],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

ax = cbars.plotcolorbar(fig, 1e-7,2e-6,4e-6,'Q (kg/kg)', x1[0],x2[1],y1[2]-0.03,y1[2]-0.02,cmap='precip_nowhite')

fig.savefig('chemistry_impact.png', bbox_inches='tight', facecolor='white')

In [ ]:
fname = runs_MT['#288']
dat288 = xr.open_dataset(basepath+fname+'.nc')
dat288 = read.fixcesmtime(dat288)
dat288zm = dat288.mean('lon')
dat288_tropics = avg.cosweightlat(dat288zm.Q, -10, 10)

fname = runs_MT['#310']
dat310 = xr.open_dataset(basepath+fname+'.nc')
dat310 = read.fixcesmtime(dat310)
dat310zm = dat310.mean('lon')
dat310_tropics = avg.cosweightlat(dat310zm.Q, -10, 10)

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = fig.add_axes([0.05,0.7,0.45,0.3])
ax.set_title('Q, 10S-10N, 30hPa', fontsize=16)
ax.set_xlabel('Month after initialization', fontsize=14)

ax.plot(np.arange(0,dat288_tropics.time.size,1), dat288_tropics.sel(lev=30, method='nearest'), label='#288')
ax.plot(np.arange(0,dat310_tropics.time.size,), dat310_tropics.sel(lev=30, method='nearest'), label='#310')

ax.legend()

In [ ]:
runs={'#288 (MT, effgw=0.7)':'f.cam6_3_112.FMTHIST_v0c.ne30.non-ogw-ubcT-effgw0.7.001'}

alldat=[]
runname=[]
for key in runs:
    print(key)
    print(runs[key])
    dat = xr.open_dataset(basepath+runs[key]+'.nc')
    dat = read.fixcesmtime(dat)
    datzm = dat.mean('lon')
    dat_tropics = avg.cosweightlat(datzm.Q, -10, 10)
    dat_mon = dat_tropics.groupby('time.month').mean('time').load()
    alldat.append(dat_mon)
    runname.append(key)
    
runname_MT=xr.DataArray(runname, dims='run', coords=[np.arange(0,len(runname),1)], name='run')
alldat_concat_MT = xr.concat(alldat, dim=runname_MT)


runs={'#259 (LT, effgw=0.3)':'f.cam6_3_112.FLTHIST_v0c.ne30.non-ogw-ubcT-effgw0.3.001'}

alldat=[]
runname=[]
for key in runs:
    print(key)
    print(runs[key])
    dat = xr.open_dataset(basepath+runs[key]+'.nc')
    dat = read.fixcesmtime(dat)
    datzm = dat.mean('lon')
    dat_tropics = avg.cosweightlat(datzm.Q, -10, 10)
    dat_mon = dat_tropics.groupby('time.month').mean('time').load()
    alldat.append(dat_mon)
    runname.append(key)
    
runname_LT=xr.DataArray(runname, dims='run', coords=[np.arange(0,len(runname),1)], name='run')
alldat_concat_LT = xr.concat(alldat, dim=runname_LT)


In [ ]:
fig = plt.figure(figsize=(16,16))

ax = cam7.plot_pre_mon(fig, mls, mls.lev, 1e-7,1.5e-6,3e-6,'MLS',x1[0],x2[0],y1[0],y2[0],cmap='precip_nowhite', paxis='lev',
                       taxis='month')
ax = cam7.plot_pre_mon(fig, era5.Q, era5.pre, 1e-7,1.5e-6,3e-6,'ERA5',x1[1],x2[1],y1[1],y2[1], cmap='precip_nowhite', paxis='pre',
                       taxis='month')

ax = cam7.plot_pre_mon(fig, alldat_concat_MT.isel(run=0), alldat_concat_MT.isel(run=0).lev, 1e-7,1.5e-6,3e-6,alldat_concat_MT.run.isel(run=0).values,
                           x1[2],x2[2],y1[2],y2[2],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

ax = cam7.plot_pre_mon(fig, alldat_concat_LT.isel(run=0), alldat_concat_LT.isel(run=0).lev, 1e-7,1.5e-6,3e-6,alldat_concat_LT.run.isel(run=0).values,
                           x1[3],x2[3],y1[3],y2[3],cmap='precip_nowhite', paxis='lev',
                           taxis='month')
    
ax = cbars.plotcolorbar(fig, 1e-7,1.5e-6,3e-6,'Q (kg/kg)', x1[1],x2[3],y1[3]-0.03,y1[3]-0.02,cmap='precip_nowhite')

fig.savefig('h2o_MT_vs_LT.png', bbox_inches='tight', facecolor='white')

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = cam7.plot_pre_mon(fig, mls, mls.lev, 1e-7,2e-6,4e-6,'MLS',x1[0],x2[0],y1[0],y2[0],cmap='precip_nowhite', paxis='lev',
                       taxis='month')
ax = cam7.plot_pre_mon(fig, era5.Q, era5.pre, 1e-7,2e-6,4e-6,'ERA5',x1[1],x2[1],y1[1],y2[1], cmap='precip_nowhite', paxis='pre',
                       taxis='month')

ax = cam7.plot_pre_mon(fig, alldat_concat_MT.isel(run=0), alldat_concat_MT.isel(run=0).lev, 1e-7,2e-6,4e-6,alldat_concat_MT.run.isel(run=0).values,
                           x1[2],x2[2],y1[2],y2[2],cmap='precip_nowhite', paxis='lev',
                           taxis='month')

ax = cam7.plot_pre_mon(fig, alldat_concat_LT.isel(run=0), alldat_concat_LT.isel(run=0).lev, 1e-7,2e-6,4e-6,alldat_concat_LT.run.isel(run=0).values,
                           x1[3],x2[3],y1[3],y2[3],cmap='precip_nowhite', paxis='lev',
                           taxis='month')
    
ax = cbars.plotcolorbar(fig, 1e-7,2e-6,4e-6,'Q (kg/kg)', x1[1],x2[3],y1[3]-0.03,y1[3]-0.02,cmap='precip_nowhite')

fig.savefig('h2o_MT_vs_LT_appropriate_ci.png', bbox_inches='tight', facecolor='white')

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = cam7.plot_pre_mon(fig, dat1zm_mon, dat1zm_mon.lev, 2e-7,2e-6,5e-6,runnum[0], x1[5],x2[5],y1[5],y2[5],cmap='precip_nowhite', paxis='lev',
                       taxis='month')

ax = cam7.plot_pre_mon(fig, dat2zm_mon, dat2zm_mon.lev, 2e-7,2e-6,5e-6,runnum[1],x1[6],x2[6],y1[6],y2[6],cmap='precip_nowhite', paxis='lev',
                       taxis='month')

ax = cam7.plot_pre_mon(fig, dat3zm_mon, dat2zm_mon.lev, 2e-7,2e-6,5e-6,runnum[2],x1[7],x2[7],y1[7],y2[7],cmap='precip_nowhite', paxis='lev',
                       taxis='month')

#ax = cbars.plotcolorbar(fig, 1e-7,1.5e-6,3e-6,'Q (kg/kg)',x1[5]+0.2,x2[9]-0.2,y1[9]-0.03,y1[9]-0.02,cmap='precip_nowhite')
ax = cbars.plotcolorbar(fig, 2e-7,2e-6,5e-6,'Q (kg/kg)', x1[5],x2[7],y1[9]-0.03,y1[9]-0.02,cmap='precip_nowhite')

In [ ]:
mlstest = xr.open_dataset("/glade/u/home/islas/python/CAM7/tau_0_ubc/datasets/Q_tropics_monthly_MLS.nc")

In [ ]:
print(mlstest)